In [1]:
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, RMSprop
from logger import logger
from resnet50 import ResNet50

Using Theano backend.


In [53]:
img_width = 224
img_height = 224
train_dir = '../data/images_clothes/train_subset'
val_dir = '../data/images_clothes/val_subset'

In [3]:
# create the base pre-trained model
base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=None)

/Users/eugeneyan/.virtualenvs/datagene/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/Users/eugeneyan/.virtualenvs/datagene/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/Users/eugeneyan/.virtualenvs/datagene/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')
/Users/eugeneyan/.virtualenvs/datagene/lib/python2.7/site-packages/keras/backend/theano_backend.py:1505: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='average_

K.image_dim_ordering: th


/Users/eugeneyan/.virtualenvs/datagene/lib/python2.7/site-packages/keras/backend/theano_backend.py:1505: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='average_exc_pad')
/Users/eugeneyan/.virtualenvs/datagene/lib/python2.7/site-packages/keras/backend/theano_backend.py:1505: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='average_exc_pad')


In [4]:
# Create top block
x = base_model.output
x = Flatten(name='flatten')(x)
x = Dense(512, activation='relu', init='glorot_uniform')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu', init='glorot_uniform')(x)
x = Dropout(0.5)(x)
pred_layer = Dense(output_dim=65, activation='softmax')(x)
model = Model(input=base_model.input, output=pred_layer)

In [5]:
# for i, layer in enumerate(model.layers):
#     print i, layer.name

In [54]:
def load_pretrained_model(model_name, output_classes, weights_path='original'):
    
    # Load base model
    if model_name == 'resnet50' and weights_path == 'original':
        base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=None)
        logger.info('Base model loaded with original weights')
    if model_name == 'resnet50' and weights_path != 'original':
        base_model = ResNet50(include_top=False, weights=None, input_tensor=None)
        weights_path = weights_path
        logger.info('Base model loaded with weights in {}'.format(weights_path))
    
    # Create top block
    x = base_model.output
    x = Flatten(name='flatten')(x)
    x = Dense(512, activation='relu', init='glorot_uniform', name='relu_1')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu', init='glorot_uniform', name='relu_2')(x)
    x = Dropout(0.5)(x)
    pred_layer = Dense(output_dim=output_classes, activation='softmax', name='softmax_output')(x)

    # Create overall model
    model = Model(input=base_model.input, output=pred_layer)
    
    # Load weights if not original weights
    if weights_path != 'original':
        model.load_weights(weights_path)
    
    return model

In [55]:
model = load_pretrained_model('resnet50', 8)
# model = load_pretrained_model('resnet50', '../data/images_clothes/model/resnet_finetuned.h5')

2016-11-23 17:09:21,921 - Base model loaded with original weights
INFO:__log__:Base model loaded with original weights


K.image_dim_ordering: th


In [56]:
conv_block = {'resnet50': {0: 176, 1: 164, 2: 154, 3: 142, 4: 132, 5: 122, 6: 112, 7: 102, 8: 92, 9: 80}}

In [65]:
def set_trainable_blocks(model_name, model, blocks=2):
    """
    Sets the number of trainable blocks, starting from the end.
    The pred layer is considered as block 0.
    """
    for layer in model.layers[:conv_block[model_name][blocks]]:
        layer.trainable = False
    for layer in model.layers[conv_block[model_name][blocks]:]:
        layer.trainable = True
        
    return model

In [66]:
model = set_trainable_blocks('resnet50', model, 1)

In [68]:
def compile_model(model):
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy', 'top_k_categorical_accuracy'])

In [34]:
def create_train_datagen(train_dir, batch_size, img_width, img_height):
    datagen = ImageDataGenerator(rescale=1./255,
                                 rotation_range=15,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)
    
    train_datagen = datagen.flow_from_directory(train_dir,
                                                target_size=(img_width, img_height),
                                                batch_size=38,
                                                class_mode='categorical',
                                                shuffle=True,
                                                seed=1368)
    
    return train_datagen

In [35]:
def create_validation_datagen(val_dir, batch_size, img_width, img_height):
    datagen = ImageDataGenerator(rescale=1./255)
    
    val_datagen = datagen.flow_from_directory(val_dir,
                                              target_size=(img_width, img_height),
                                              batch_size=38,
                                              class_mode='categorical',
                                              shuffle=True,
                                              seed=1368)
    
    return val_datagen

In [36]:
train_generator = create_train_datagen(train_dir, 38, img_width, img_height)

Found 7200 images belonging to 8 classes.


In [37]:
validation_generator = create_validation_datagen(val_dir, 38, img_width, img_height)

Found 760 images belonging to 8 classes.


In [38]:
def fit_model(model, train_generator, validation_generator, epoches):
    model.fit_generator(train_generator, samples_per_epoch=train_generator.N, nb_epoch=epoches,
                        validation_data=validation_generator,
                        nb_val_samples=validation_generator.N,
                        max_q_size=20, nb_worker=4)

In [69]:
compile_model(model)

In [47]:
fit_model(model, train_generator, validation_generator, 1)

Epoch 1/1


KeyboardInterrupt: 